## Author : Priteshkumar Gohil

In [4]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import torch.utils.data as data
from torchvision import transforms

import os
import os.path
import sys
if('/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path):
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
import numpy as np
from scipy import signal
from PIL import Image

In [5]:
# base class for dataloader vision.py

class VisionDataset(data.Dataset):
    _repr_indent = 4

    def __init__(self, root, transforms=None, transform=None, target_transform=None):
        if isinstance(root, torch._six.string_classes):
            root = os.path.expanduser(root)
        self.root = root

        has_transforms = transforms is not None
        has_separate_transform = transform is not None or target_transform is not None
        if has_transforms and has_separate_transform:
            raise ValueError("Only transforms or transform/target_transform can "
                             "be passed as argument")

        # for backwards-compatibility
        self.transform = transform
        self.target_transform = target_transform

        if has_separate_transform:
            transforms = StandardTransform(transform, target_transform)
        self.transforms = transforms

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __repr__(self):
        head = "Dataset " + self.__class__.__name__
        body = ["Number of datapoints: {}".format(self.__len__())]
        if self.root is not None:
            body.append("Root location: {}".format(self.root))
        body += self.extra_repr().splitlines()
        if hasattr(self, "transforms") and self.transforms is not None:
            body += [repr(self.transforms)]
        lines = [head] + [" " * self._repr_indent + line for line in body]
        return '\n'.join(lines)

    def _format_transform_repr(self, transform, head):
        lines = transform.__repr__().splitlines()
        return (["{}{}".format(head, lines[0])] +
                ["{}{}".format(" " * len(head), line) for line in lines[1:]])

    def extra_repr(self):
        return ""


class StandardTransform(object):
    def __init__(self, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform

    def __call__(self, input, target):
        if self.transform is not None:
            input = self.transform(input)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return input, target

    def _format_transform_repr(self, transform, head):
        lines = transform.__repr__().splitlines()
        return (["{}{}".format(head, lines[0])] +
                ["{}{}".format(" " * len(head), line) for line in lines[1:]])

    def __repr__(self):
        body = [self.__class__.__name__]
        if self.transform is not None:
            body += self._format_transform_repr(self.transform,
                                                "Transform: ")
        if self.target_transform is not None:
            body += self._format_transform_repr(self.target_transform,
                                                "Target transform: ")

        return '\n'.join(body)

In [9]:
def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.
    Args:
        filename (string): path to a file
        extensions (tuple of strings): extensions to consider (lowercase)
    Returns:
        bool: True if the filename ends with one of given extensions
    """
    return filename.lower().endswith(extensions)


def is_image_file(filename):
    """Checks if a file is an allowed image extension.
    Args:
        filename (string): path to a file
    Returns:
        bool: True if the filename ends with a known image extension
    """
    return has_file_allowed_extension(filename, IMG_EXTENSIONS)


def make_dataset(dir, class_to_idx, extensions=None, is_valid_file=None):
    images = []
    # dir = /media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/
    dir = os.path.expanduser(dir)
    if not ((extensions is None) ^ (is_valid_file is None)):
        raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")
    if extensions is not None:
        def is_valid_file(x):
            return has_file_allowed_extension(x, extensions)
    for target in sorted(class_to_idx.keys()):
        # d = /media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            # root = /media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez
            #        /media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez/100_432
            #        /media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez/100_432/right
            #        /media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez/100_432/right/0
            # fnames = []
            # fnames = []
            # fnames = []
            # fnames = ['front_0.jpg', 'front_1.jpg', 'front_depth.mp4', 'front_depth0.mp4', 'front_rgb.mp4', 
            #           'label.txt', 'left_0.jpg', 'left_1.jpg', 'left_rgb.mp4', 'pos.txt', 'tactile.txt']
            image_path = None
            tactile_path = None
            label_path = None
            video_path = None
            left_video_path = None
            pos_path = None
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                if(path.lower().endswith(extensions[0])):
                    image_path = path
                elif(path.lower().endswith(extensions[1])):
                    tactile_path = path
                elif(path.lower().endswith(extensions[2])):
                    label_path = path
                elif(path.lower().endswith(extensions[3])):
                    video_path = path
                elif(path.lower().endswith(extensions[4])):
                    left_video_path = path
                elif(path.lower().endswith(extensions[5])):
                    pos_path = path
            # return data iff it contains all the 4 files.
            # If you wants to return irrespective of video modify the code
            if((image_path and tactile_path and label_path and video_path and left_video_path and pos_path) is not None):
                item = (image_path,tactile_path,label_path,video_path, left_video_path,pos_path, np.loadtxt(label_path)[3])
                images.append(item)
    return images


class VisualTactile(VisionDataset):
    """A generic data loader where the samples are arranged in this way: ::
        root/class_x/xxx.ext
        root/class_x/xxy.ext
        root/class_x/xxz.ext
        root/class_y/123.ext
        root/class_y/nsdf3.ext
        root/class_y/asd932_.ext
    Args:
        root (string): Root directory path.
        loader (callable): A function to load a sample given its path.
        extensions (tuple[string]): A list of allowed extensions.
            both extensions and is_valid_file should not be passed.
        transform (callable, optional): A function/transform that takes in
            a sample and returns a transformed version.
            E.g, ``transforms.RandomCrop`` for images.
        target_transform (callable, optional): A function/transform that takes
            in the target and transforms it.
        is_valid_file (callable, optional): A function that takes path of an Image file
            and check if the file is a valid_file (used to check of corrupt files)
            both extensions and is_valid_file should not be passed.
     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        samples (list): List of (sample path, class_index) tuples
        targets (list): The class_index value for each image in the dataset
    """

    def __init__(self, root, loader, extensions=None, transform=None, target_transform=None, is_valid_file=None):
        super(VisualTactile, self).__init__(root)
        self.transform = transform
        self.target_transform = target_transform

        # classes = name of the folder
        # class_to_idx = dictionary with class name(folder name) and index
        classes, class_to_idx = self._find_classes(self.root)
        samples = make_dataset(self.root, class_to_idx, extensions, is_valid_file)
#         print(samples)
        if len(samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                "Supported extensions are: " + ",".join(extensions)))

        self.loader = loader
        self.extensions = extensions

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]

    def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        image_path, tactile_path, label_path, video_path, left_video_path, pos_path, target = self.samples[index]
        sample_image = self.loader(image_path)
        sample_tactile = tactile_loader(tactile_path)
        sample_lable = label_loader(label_path)
        sample_video = vid_loader(video_path)
        sample_left_video = vid_loader(left_video_path)
        sample_pos = tactile_loader(pos_path)
        if self.transform is not None:
            sample_image = self.transform(sample_image)
            sample_tactile = self.transform(sample_tactile)
            sample_lable = self.transform(sample_lable)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample_image, sample_tactile, sample_lable, sample_video, sample_left_video, sample_pos, target
#         return sample_image, sample_tactile, sample_lable, target

    def __len__(self):
        return len(self.samples)


# IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
# Select only these items in dataset
# IMG_EXTENSIONS = ('front_0.jpg')
IMG_EXTENSIONS = ('front_0.jpg','tactile.txt','label.txt','front_rgb.mp4','left_rgb.mp4', 'pos.txt')
# TAC_EXTENSIONS = ('tactile.txt')
# LAB_EXTENSIONS = ('label.txt')
# VID_EXTENSIONS = ('front_rgb.mp4')

def tactile_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    up_samples=24*18
    with open(path, 'rb') as f:
        tactile_frame = pd.read_csv(f,delimiter=' ', header=None)
        tactile = tactile_frame.as_matrix()
        tactile = tactile.astype('float')
        tactile = signal.resample(tactile,up_samples)
        return tactile #to get samples equal to the video frames, make length variable in vid_loader global and
                        #then tactile[:lenth]

def label_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        return np.loadtxt(f)

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def vid_loader(path):
    print(path)
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    count = 0
    required_frames = 64
    frame_output = 5
    # 18x18 = 324  frames in total 
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("length",length)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    print("FPS", fps )
    duration = length / fps
    print("duration",duration)

    images = []
    while success:
        if((count%5)==0 and (len(images)<required_frames)):
#             images.append(image)
            images.append(Image.fromarray(cv2.cvtColor(image,cv2.COLOR_BGR2RGB)))#for PIL RGB image
        success,image = vidcap.read()
        count += 1
    missing_frames = required_frames - len(images)
    if(missing_frames):
        remove_element = (missing_frames//frame_output)+1
        del images[-remove_element:]
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-missing_frames-1)
        success,image = vidcap.read()
        while success:
#                 images.append(image) #BGR Image
#                 images.append(cv2.cvtColor(image,cv2.COLOR_BGR2RGB)) #for RGB image
            images.append(Image.fromarray(cv2.cvtColor(image,cv2.COLOR_BGR2RGB)))#for PIL RGB image
            success,image = vidcap.read()
    return images

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)


class ImageFolder(VisualTactile):
    """A generic data loader where the images are arranged in this way: ::
        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png
        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png
    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
        is_valid_file (callable, optional): A function that takes path of an Image file
            and check if the file is a valid_file (used to check of corrupt files)
     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader, is_valid_file=None):
        super(ImageFolder, self).__init__(root, loader, IMG_EXTENSIONS if is_valid_file is None else None,
                                          transform=transform,
                                          target_transform=target_transform,
                                          is_valid_file=is_valid_file)
        self.imgs = self.samples

In [10]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/"
tv_dataset = ImageFolder(root=path)
# a=tv_dataset[150] 
# a[3]
tv_dataset[150][1].shape
# tv_dataset[150][4][20]
# for i in a:
#     cv2.imshow("im", a)
#     cv2.waitKey(200)
# cv2.destroyAllWindows()
# a[0].shape
## or you can use imagefolder to see data
# tv_dataset.samples[100]

## To check the to

/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Coffeecup/100_843/back/0/front_rgb.mp4
length 339
FPS 18
duration 18.833333333333332
/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Coffeecup/100_843/back/0/left_rgb.mp4
length 339
FPS 18
duration 18.833333333333332


(432, 16)